In [2]:
import numpy as np
import pandas as pd

import requests, re, os
import urllib
from bs4 import BeautifulSoup as bs

#### Make Function to Crawl opioid dispense table

In [77]:
def make_table(years):
    new_df = pd.DataFrame()

    for year in years:
        print(year)
        html = requests.get('https://www.cdc.gov/drugoverdose/maps/rxcounty{}.html'.format(year))
        print(html)
        soup = bs(html.text, 'html.parser')
        html.close()

        if html.status_code == 200:
            try:
                table = soup.find('table', {'class':'table table-striped'})

                thead = table.find('thead')

                ths = thead.find_all('th')
                th_list = [i.get_text() for i in ths]

                tbody = table.find('tbody')
                trs = tbody.find_all('tr')
                tr_list = [i.get_text(';', strip=True) for i in trs]


                table_content_df = pd.DataFrame(data = tr_list)

                table_content_df = pd.DataFrame(data = table_content_df[0].str.split(';', 4).tolist(),
                                            columns = th_list)
                table_content_df['year'] = year
    

            except:
                print("Table not scrapable")
                table_content_df = pd.DataFrame()
                
            new_df = new_df.append(table_content_df)
            
        else:
            print("HTML does not open")
            pass
        
    return new_df

In [79]:
years = list(range(2010,2020))
opioid_dispense_df = make_table(years)

2010
<Response [200]>
2011
<Response [200]>
2012
<Response [200]>
2013
<Response [200]>
2014
<Response [200]>
2015
<Response [200]>
2016
<Response [200]>
2017
<Response [200]>
2018
<Response [200]>
2019
<Response [200]>


In [84]:
# Clean File
opioid_dispense_df['Opioid Dispensing Rate per 100'] = opioid_dispense_df['Opioid Dispensing Rate per 100'].replace({'–':np.nan})
opioid_dispense_df = opioid_dispense_df.reset_index(drop=True)
# Safe File
opioid_dispense_df.to_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Opioid/opioid_dispense.csv', index=False)

In [112]:
opioid_dispense_df.head()

,County,State,County FIPS Code,Opioid Dispensing Rate per 100,year
0,"Aleutians East, AK",AK,02013,NaN,2010
1,"Aleutians West, AK",AK,02016,NaN,2010
2,"Anchorage, AK",AK,02020,75.1,2010
3,"Bethel, AK",AK,02050,NaN,2010
4,"Bristol Bay, AK",AK,02060,NaN,2010


##### Clean Drug Overdose Death File

In [91]:
# Open Text File (Tab-Delimited)
import csv
with open(r'/Users/krhee/Downloads/Underlying Cause of Death, 1999-2019 (2).txt') as f:
    reader = csv.reader(f, delimiter="\t")
    df = list(reader)
    
death_df = pd.DataFrame(data=df)

In [95]:
# Clean table
death_df.columns = death_df.iloc[0].values
death_df = death_df.drop([0])
death_df = death_df.drop(columns=['Notes'])

death_df = death_df.dropna(subset=['County','County Code'])

# Save File
death_df.to_csv(r'/Users/krhee/Dropbox (ASU)/RA_2021/Datasets/Opioid/underlying_cause_of_death_drug_overdose.csv', index=False)

In [113]:
death_df.head()

,County,County Code,Year,Year Code,ICD-10 113 Cause List,ICD-10 113 Cause List Code,Deaths,Population,Crude Rate
1,"Baldwin County, AL",01003,2017,2017,"#Accidents (unintentional injuries) (V01-X59,Y...",GR113-112,23,212628,10.8
2,"Baldwin County, AL",01003,2017,2017,"Nontransport accidents (W00-X59,Y86)",GR113-117,23,212628,10.8
3,"Baldwin County, AL",01003,2017,2017,Accidental poisoning and exposure to noxious s...,GR113-122,23,212628,10.8
4,"Baldwin County, AL",01003,2018,2018,"#Accidents (unintentional injuries) (V01-X59,Y...",GR113-112,24,218022,11.0
5,"Baldwin County, AL",01003,2018,2018,"Nontransport accidents (W00-X59,Y86)",GR113-117,24,218022,11.0
